In [1]:
using DifferentialEquations
using Plots
using LinearAlgebra
import ForwardDiff
import DiffResults
using DataFrames
using AstrodynamicsBase
# import joptimise
using Printf
using JSON
using CSV
plotly()

┌ Warning: For saving to png with the `Plotly` backend `PlotlyBase` and `PlotlyKaleido` need to be installed.
│   err = ArgumentError("Package PlotlyBase not found in current path:\n- Run `import Pkg; Pkg.add(\"PlotlyBase\")` to install the PlotlyBase package.\n")
└ @ Plots C:\Users\yujit\.julia\packages\Plots\gzYVM\src\backends.jl:420


Plots.PlotlyBackend()

In [2]:
include("../../julia-r3bp/R3BP/src/R3BP.jl")
include("../src/SailorMoon.jl")   # relative path to main file of module

Main.SailorMoon

In [3]:
param3b = SailorMoon.dyanmics_parameters()
lps = SailorMoon.lagrange_points(param3b.mu2)

5×6 Matrix{Float64}:
  0.836915   0.0       0.0  0.0  0.0  0.0
  1.15568    0.0       0.0  0.0  0.0  0.0
 -1.00506    0.0       0.0  0.0  0.0  0.0
  0.487849   0.866025  0.0  0.0  0.0  0.0
  0.487849  -0.866025  0.0  0.0  0.0  0.0

In [4]:
lp = 2
Az_km = 1200.0
println("Halo guess Az_km: $Az_km")
northsouth = 3   # 1 or 3
guess0 = R3BP.halo_analytical_construct(param3b.mu2, lp, Az_km, param3b.lstar, northsouth)
res = R3BP.ssdc_periodic_xzplane([param3b.mu2,], guess0.x0, guess0.period, fix="period")
res.flag

Halo guess Az_km: 1200.0


1

In [5]:
x0_stm = vcat(res.x0, reshape(I(6), (6^2,)))[:]
prob_cr3bp_stm = ODEProblem(R3BP.rhs_cr3bp_svstm!, x0_stm, res.period, (param3b.mu2))
sol = solve(prob_cr3bp_stm, Tsit5(), reltol=1e-12, abstol=1e-12)#, saveat=LinRange(0, period, n+1))
monodromy = R3BP.get_stm(sol, 6)   # get monodromy matrix
ys0 = R3BP.get_eigenvector(monodromy, true, 1) # monodromy eigenvector


Linear stability ν = 618.7618470919092


6-element Vector{Float64}:
  0.2672179149228554
  0.2793813746152613
  2.469616446417975e-26
 -0.7710131455619899
 -0.5060428468351018
  1.3395041586794771e-25

In [8]:
function rhs_bcr4bp_thrust!(du,u,p,t)
    # unpack arguments
    μ, μ_3, t0, a_s, ω_s, τ, θ, β, mdot, tmax = p
    # decompose state
    x, y, z, vx, vy, vz, mass = u

    # calculate radii
    r1 = sqrt( (x+μ)^2 + y^2 + z^2 )
    r2 = sqrt( (x-1+μ)^2 + y^2 + z^2 )

    # sun position
    xs = a_s*cos(ω_s*t + t0)
    ys = a_s*sin(ω_s*t + t0)
    zs = 0.0
    r3 = sqrt( (x-xs)^2 + (y-ys)^2 + (z-zs)^2 )

    # compute delta-V vector
    dir_v = dv_lvlh2inertial(u[1:6], [τ, θ, β])
    ts = tmax*dir_v

    # position-state derivative
    du[1] = vx
    du[2] = vy
    du[3] = vz

    # velocity derivatives
    du[4] =  2*vy + x - ((1-μ)/r1^3)*(μ+x)  + (μ/r2^3)*(1-μ-x)  + ( -(μ_3/r3^3)*(x-xs) - (μ_3/a_s^3)*xs ) + ts[1]/mass
    du[5] = -2*vx + y - ((1-μ)/r1^3)*y      - (μ/r2^3)*y        + ( -(μ_3/r3^3)*(y-ys) - (μ_3/a_s^3)*ys ) + ts[2]/mass
    du[6] =           - ((1-μ)/r1^3)*z      - (μ/r2^3)*z        + ( -(μ_3/r3^3)*(z)    - (μ_3/a_s^3)*zs ) + ts[3]/mass
    
    println(((1-μ)/r1^3)*(μ+x), " ", (μ/r2^3)*(1-μ-x), " ",  -(μ_3/r3^3)*(x-xs) - (μ_3/a_s^3)*xs)

    # mass derivative
    du[7] = -mdot*τ
end

rhs_bcr4bp_thrust! (generic function with 1 method)

In [9]:
function ensemble_grid_search(ϕ0::Float64, ϵr::Float64, ϵv::Float64, θf::Float64) where T
    
    tof_bck = 22
    βf = π - θf
    
    flag = 1   # exit flag: terminated -> 1, completed -> 0
    
    # arrival LPO object
    LPOArrival = SailorMoon.CR3BPLPO2(
        res.x0, res.period, ys0, prob_cr3bp_stm, ϵr, ϵv, Tsit5(), 1e-12, 1e-12
    );
    
    svf = vcat(SailorMoon.set_terminal_state2(ϕ0, θf, param3b, LPOArrival), 1.0)
    println(svf)
    params = [param3b.mu2, param3b.mus, βf, param3b.as, param3b.oms, 0.0, 0.0, 0.0, 0.0, 0.0]
    tspan = [0, -tof_bck]
    prob_bck = ODEProblem(rhs_bcr4bp_thrust!, svf, tspan, params)
    sol_bck = solve(prob_bck, Tsit5(), reltol=1e-12, abstol=1e-12);
    
    if sol.retcode == :Terminated
        flag = 0
    end

    ra = param[11]
    rp  = param[12]
    tof = sol_bck.t[end]
    
    return flag, rp, ra, tof, sol_bck
end

ensemble_grid_search (generic function with 1 method)

In [11]:
# ---------- ensemble simulation ---------- #
function prob_func(prob, i, repeat)
    print("\rproblem # $i / $nic")
    remake(prob, u0=x0s[i], tspan=(0.0, 2.0tfs[i]))
end

ensemble_prob = EnsembleProblem(prob, prob_func=prob_func)
sim = solve(ensemble_prob, Tsit5(), EnsembleThreads(), trajectories=length(x0s), callback=cbs, 
    reltol=1e-12, abstol=1e-12);

LoadError: UndefVarError: prob not defined

In [ ]:
# terminate condition: hit earth or moon
function terminate_affect!(int)
    terminate!(int)
end

function terminate_condition(u,t,int)
    # Hit earth
    cond1 = sqrt((u[1] - param3b.mu2) ^2 + u[2]^2 + u[3]^2) - (6600 / param3b.lstar)
    # Hit moon
    cond2 = sqrt((u[1] - (1-param3b.(mu2)) ^2 + u[2]^2 + u[3]^2) - (1770 / param3b.lstar)
    return - cond1 * cond2
end

terminate_cb = ContinuousCallback(terminate_condition,terminate_affect!)

In [ ]:
# store the apoapsis value
function apoapsis_cond(u,t,int)
    # condition 1: SC is sufficiently far from the moon
    cond1 = sqrt((u[1] - (1-param3b.mu2))^2 + u[2]^2 + u[3]^2) - 5000 / param3b.lstar 
    
    # condition 2: dot product of velocity and position is zero
    cond2 = dot(u[1:3], u[4:6]) == 0
    
    return cond1 * cond2
end

function apoapsis_affect!(int)
    return NaN
end
apoapsis_cb = ContinuousCallback(apoapsis_cond, apoapsis_affect!; rootfind=true, save_positions=(true,false))

In [ ]:
# store the periapsis value and terminate
function periapsis_cond(u,t,int)
    # 0.95 * moon sma < SC distantce from B1 < 1.05 * moon sma
    cond1 = sqrt(u[1]^2 + u[2]^2 + u[3]^2) - (1 - param3b.mu2)*0.95 
    cond2 = - sqrt(u[1]^2 + u[2]^2 + u[3]^2) + (1 - param3b.mu2)*1.05
    return cond1 * cond2
end

function periapsis_affect!(int)
    terminate!(int)
end
periapsis_cb = ContinuousCallback(apoapsis_cond, apoapsis_affect!)

In [ ]:
cbs = CallbackSet(terminate_cb, apoapsis_cb, periapsis_cb)

In [90]:
# Grid search parameters: CHANGE HERE
ϕ_vec    = [0.0]    #range(0, stop=2*pi, length=20)
epsr_vec = [1e-6] #10 .^(-12:-6)
epsv_vec = [1e-6] #10 .^(-12:-6)
θ_vec    = [0.0]   #range(0, stop=2*pi, length=20)

1-element Vector{Float64}:
 0.0

In [91]:
entries = ["flag", "phi0", "epsr", "epsv", "thetaf", "rp", "ra", "tof"]
df = DataFrame([ name =>[] for name in entries])

nodes = []

# make initial conditions 
for ϕ0 in ϕ_vec
    for ϵr in epsr_vec
        for ϵv in epsv_vec
            for θf in θ_vec
                
                push!(nodes, [ϕ0, ϵr, ϵv, θf])
                
            end
        end
    end
end


flag, rp, ra, tof, sol = propagate_grid_search(ϕ0, ϵr, ϵv, θf)
push!(df, [flag, ϕ0, ϵr, ϵv, θf, rp, ra, tof])

print(df)
# extract only successfully exited trajectories
# filter(:flag => ==(1), df)
# CSV.write("grid_search.csv", df)

pcart = plot(size=(700,500), frame_style=:box, aspect_ratio=:equal, grid=0.4)
plot!(Array(sol)[1,:], Array(sol)[2,:], color=:purple, linewidth=1.5, label="bck")

[2.6721791492285546e-7, 2.7938137461526135e-7, 2.469616446417975e-32, -7.710131455619899e-7, -5.060428468351019e-7, 1.3395041586794773e-31]
[1.1226882532375977, 2.7938137461526135e-7, -5.228883154550508e-26, -7.710131455619899e-7, 0.16509880733691906, 1.3395041586794773e-31, 1.0]
1×8 DataFrame
 Row │ flag  phi0  epsr    epsv    thetaf  rp   ra   tof 
     │ Any   Any   Any     Any     Any     Any  Any  Any 
─────┼───────────────────────────────────────────────────
   1 │ 1.0   0.0   1.0e-6  1.0e-6  0.0     1.0  1.0  1.0

In [92]:
param3b.lstar

384748.32292972936